In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu_devices))
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

Num GPUs Available:  1


In [3]:
from bert.dataset import create_masked_input_dataset

In [4]:
!nvidia-smi

Tue Feb  4 06:00:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.26       Driver Version: 430.26       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Quadro GV100        Off  | 00000000:37:00.0 Off |                  Off |
| 33%   46C    P0    46W / 250W |    234MiB / 32508MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## BERT layers

In [5]:
from bert.dataset import create_masked_input_dataset

training_data = create_masked_input_dataset(
    sequence_path='../uniparc_data/train_uniref100.txt.gz',
    max_sequence_length=1024,
    fix_sequence_length=True,
    batch_size=2)

training_data = training_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)

valid_data = create_masked_input_dataset(
    sequence_path='../uniparc_data/dev_uniref50.txt.gz',
    max_sequence_length=1024,
    batch_size=2)

valid_data = valid_data.repeat().prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
masked_seqs, true_values = next(iter(valid_data))

In [7]:
true_values.numpy().max()

20

In [8]:
from tensorflow.keras import layers

from bert.layers import (PositionEmbedding, Transformer, TokenEmbedding, Bias,
                         gelu, initializer, Projection, DenseNoMask)

In [12]:
vocab_size = 24
embedding_dimension = 32
model_dimension = 128
transformer_dimension = 4 * model_dimension
num_attention_heads = model_dimension // 64
num_transformer_layers = 4
dropout_rate = 0.

inputs = layers.Input(shape=(None,), dtype=tf.int32, batch_size=None)

# Amino-acid level embeddings
embeddings = layers.Embedding(
    vocab_size, model_dimension, embeddings_initializer=initializer(),
    mask_zero=True)(inputs)

# Initialize transformer, use ALBERT-style weight sharing
transformer = Transformer(
    num_attention_heads, transformer_dimension,
    attention_type='relative', max_relative_position=10,
    dropout=dropout_rate)

# Stack transformers together
for i in range(num_transformer_layers):
    embeddings = transformer(embeddings)

# Project back to original embedding dimension
out = DenseNoMask(21, activation=gelu,
                  kernel_initializer=initializer())(embeddings)

model = tf.keras.Model(inputs, out, name='model')

In [13]:
from bert.optimizers import ECE, masked_sparse_categorical_crossentropy, BertLinearSchedule
    
opt = tfa.optimizers.AdamW(learning_rate=1E-4, weight_decay=0.0)

model.compile(
    loss=masked_sparse_categorical_crossentropy,
    metrics=[ECE],
    optimizer=opt,
    experimental_run_tf_function=True)

callbacks = [BertLinearSchedule(1E-4, 100, int(1E7))]

In [14]:
model.fit(training_data, steps_per_epoch=100, epochs=4,
          verbose=1, validation_data=valid_data, validation_steps=10,
          callbacks=callbacks)

Train for 100 steps, validate for 10 steps
Epoch 1/4
100/100 [==============================] - 47s 469ms/step - loss: 3.0323 - ECE: 20.7536 - val_loss: 2.9765 - val_ECE: 19.6253
Epoch 2/4
100/100 [==============================] - 39s 391ms/step - loss: 2.8898 - ECE: 18.0370 - val_loss: 2.8333 - val_ECE: 17.0245
Epoch 3/4
100/100 [==============================] - 39s 394ms/step - loss: 2.7981 - ECE: 16.4704 - val_loss: 2.7939 - val_ECE: 16.3860
Epoch 4/4
100/100 [==============================] - 39s 392ms/step - loss: 2.7714 - ECE: 16.0364 - val_loss: 2.7955 - val_ECE: 16.3871


In [ ]:
# training_data_large = create_masked_input_dataset(
#     sequence_path='../uniparc_data/sequences_train.txt',
#     max_sequence_length=512,
#     batch_size=4,
#     masking_freq=.05)

# training_data_large = training_data_large.repeat().prefetch(tf.data.experimental.AUTOTUNE)

# valid_data_large = create_masked_input_dataset(
#     sequence_path='../uniparc_data/sequences_valid.txt',
#     max_sequence_length=512,
#     batch_size=4,
#     masking_freq=.05)

# valid_data_large = valid_data_large.repeat().prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# model.fit(training_data_large, steps_per_epoch=1000, epochs=5,
#           verbose=1, validation_data=valid_data_large, validation_steps=100,
#           callbacks=callbacks)

In [ ]:
model.save('test_ckpts/test.h5')

In [ ]:
tf.keras.models.load_model('test_ckpts/test.h5')